In [1]:
import pandas as pd
import numpy as np


# Loading data and getting features overview

In [2]:
df = pd.read_csv('/Users/LinhVu/code/linhvu12/elissus/RawData/immo_data.csv')

In [3]:
#How many entries & features?

df.shape

(268850, 49)

In [4]:
df.columns

Index(['regio1', 'serviceCharge', 'heatingType', 'telekomTvOffer',
       'telekomHybridUploadSpeed', 'newlyConst', 'balcony', 'picturecount',
       'pricetrend', 'telekomUploadSpeed', 'totalRent', 'yearConstructed',
       'scoutId', 'noParkSpaces', 'firingTypes', 'hasKitchen', 'geo_bln',
       'cellar', 'yearConstructedRange', 'baseRent', 'houseNumber',
       'livingSpace', 'geo_krs', 'condition', 'interiorQual', 'petsAllowed',
       'street', 'streetPlain', 'lift', 'baseRentRange', 'typeOfFlat',
       'geo_plz', 'noRooms', 'thermalChar', 'floor', 'numberOfFloors',
       'noRoomsRange', 'garden', 'livingSpaceRange', 'regio2', 'regio3',
       'description', 'facilities', 'heatingCosts', 'energyEfficiencyClass',
       'lastRefurbish', 'electricityBasePrice', 'electricityKwhPrice', 'date'],
      dtype='object')

In [5]:
#In which states are the properties located?

df['regio1'].unique()

array(['Nordrhein_Westfalen', 'Rheinland_Pfalz', 'Sachsen', 'Bremen',
       'Schleswig_Holstein', 'Baden_Württemberg', 'Thüringen', 'Hessen',
       'Niedersachsen', 'Bayern', 'Hamburg', 'Sachsen_Anhalt',
       'Mecklenburg_Vorpommern', 'Berlin', 'Brandenburg', 'Saarland'],
      dtype=object)

## Extract Berlin

In [6]:
# Filter the DataFrame for Berlin

df_berlin = df[df['regio1'] == 'Berlin']
df_berlin.shape

(10406, 49)

In [7]:
# Save the Berlin subset to a new CSV file
df_berlin.to_csv('berlin_rawdata.csv', index=False)

In [8]:
df_berlin.head()

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,...,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
45,Berlin,320.00,central_heating,NaN,NaN,False,True,10,4.99,NaN,...,Berlin,Staaken_Spandau,Diese attraktive EG-Wohnung kann ab sofort bez...,NaN,NaN,NaN,NaN,NaN,NaN,Feb20
61,Berlin,79.00,central_heating,ONE_YEAR_FREE,NaN,False,False,17,7.35,40.0,...,Berlin,Weißensee_Weißensee,Suchen Sie eine 2-Zimmerwohnung in Weißensee? ...,Zusammengefasst bietet die Wohnung:\n+ Vollstä...,68.00,NaN,NaN,NaN,NaN,Oct19
77,Berlin,150.00,floor_heating,ONE_YEAR_FREE,NaN,True,True,15,6.60,40.0,...,Berlin,Mitte_Mitte,Dieses luxuriöse Neubauquartier direkt am Luis...,"• sehr ruhig gelegen, trotz zentraler Lage, da...",NaN,NaN,NaN,NaN,NaN,Oct19
86,Berlin,228.78,floor_heating,NaN,NaN,False,True,2,8.63,NaN,...,Berlin,Kreuzberg_Kreuzberg,Bei dieser ansprechenden Immobilie handelt es ...,Balkon und 10m Terasse,NaN,NaN,NaN,NaN,NaN,Oct19
87,Berlin,147.08,district_heating,ONE_YEAR_FREE,NaN,True,True,9,7.56,40.0,...,Berlin,Tiergarten_Tiergarten,"In the """"Wasserstadt - Mitte"""" you can expect ...",Your spacious ground floor apartment is equipp...,73.54,B,2019.0,NaN,NaN,Feb20


In [9]:
df_berlin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10406 entries, 45 to 268829
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   regio1                    10406 non-null  object 
 1   serviceCharge             10171 non-null  float64
 2   heatingType               8935 non-null   object 
 3   telekomTvOffer            8994 non-null   object 
 4   telekomHybridUploadSpeed  918 non-null    float64
 5   newlyConst                10406 non-null  bool   
 6   balcony                   10406 non-null  bool   
 7   picturecount              10406 non-null  int64  
 8   pricetrend                10381 non-null  float64
 9   telekomUploadSpeed        8993 non-null   float64
 10  totalRent                 9744 non-null   float64
 11  yearConstructed           8981 non-null   float64
 12  scoutId                   10406 non-null  int64  
 13  noParkSpaces              2013 non-null   float64
 14  firi

In [10]:
#Check on missing value
#absolute value
df_berlin.isnull().sum().sort_values(ascending=False)

#relative value
df_berlin.isnull().sum().sort_values(ascending=False)/len(df_berlin)

telekomHybridUploadSpeed    0.911782
electricityKwhPrice         0.808284
electricityBasePrice        0.808284
noParkSpaces                0.806554
lastRefurbish               0.679416
energyEfficiencyClass       0.664713
petsAllowed                 0.464924
heatingCosts                0.458293
interiorQual                0.404382
thermalChar                 0.314722
numberOfFloors              0.300500
firingTypes                 0.219489
condition                   0.217951
houseNumber                 0.146742
streetPlain                 0.146742
heatingType                 0.141361
yearConstructed             0.136940
yearConstructedRange        0.136940
telekomUploadSpeed          0.135787
telekomTvOffer              0.135691
facilities                  0.127138
floor                       0.105708
typeOfFlat                  0.077263
description                 0.063809
totalRent                   0.063617
serviceCharge               0.022583
pricetrend                  0.002402
r

In [11]:
df_berlin.regio1.unique()

array(['Berlin'], dtype=object)

In [12]:
#drop duplicates
df_berlin.drop_duplicates().reset_index()

,index,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,...,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
0,45,Berlin,320.00,central_heating,NaN,NaN,False,True,10,4.99,...,Berlin,Staaken_Spandau,Diese attraktive EG-Wohnung kann ab sofort bez...,NaN,NaN,NaN,NaN,NaN,NaN,Feb20
1,61,Berlin,79.00,central_heating,ONE_YEAR_FREE,NaN,False,False,17,7.35,...,Berlin,Weißensee_Weißensee,Suchen Sie eine 2-Zimmerwohnung in Weißensee? ...,Zusammengefasst bietet die Wohnung:\n+ Vollstä...,68.00,NaN,NaN,NaN,NaN,Oct19
2,77,Berlin,150.00,floor_heating,ONE_YEAR_FREE,NaN,True,True,15,6.60,...,Berlin,Mitte_Mitte,Dieses luxuriöse Neubauquartier direkt am Luis...,"• sehr ruhig gelegen, trotz zentraler Lage, da...",NaN,NaN,NaN,NaN,NaN,Oct19
3,86,Berlin,228.78,floor_heating,NaN,NaN,False,True,2,8.63,...,Berlin,Kreuzberg_Kreuzberg,Bei dieser ansprechenden Immobilie handelt es ...,Balkon und 10m Terasse,NaN,NaN,NaN,NaN,NaN,Oct19
4,87,Berlin,147.08,district_heating,ONE_YEAR_FREE,NaN,True,True,9,7.56,...,Berlin,Tiergarten_Tiergarten,"In the """"Wasserstadt - Mitte"""" you can expect ...",Your spacious ground floor apartment is equipp...,73.54,B,2019.0,NaN,NaN,Feb20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10401,268716,Berlin,155.00,central_heating,ONE_YEAR_FREE,NaN,False,False,9,6.90,...,Berlin,Friedrichshagen_Köpenick,"1-Zimmer-Wohnung, DG links\n\n\nGröße: 57 m² ...","gefliestes Duschbad, wandhängendes WC,\nEinba...",30.00,NaN,NaN,NaN,NaN,May19
10402,268733,Berlin,100.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,False,10,7.59,...,Berlin,Spandau_Spandau,Extras:\n- Treppenhausreinigung\n- Hausmeister...,"Allgemein:\n- Kabel-TV, Grundversorgung\n- Vor...",NaN,NaN,NaN,90.76,0.1985,Sep18
10403,268746,Berlin,200.00,district_heating,ONE_YEAR_FREE,NaN,False,True,16,8.45,...,Berlin,Waidmannslust_Reinickendorf,Die hier angebotene Mietwohnung befindet sich ...,"- Erstbezug\n- 3,5 Zimmer\n- modernisiertes, g...",130.00,F,2019.0,NaN,NaN,May19
10404,268758,Berlin,100.00,district_heating,ONE_YEAR_FREE,NaN,True,False,13,6.14,...,Berlin,Charlottenburg_Charlottenburg,Bei dem Objekt handelt es sich um einen energi...,"Das Apartment ist voll möbliert, mit einer vol...",NaN,NaN,2019.0,NaN,NaN,Feb20


# Handling missing data (Berlin dataset)

## Dropping columns/features

In [13]:
# >80%  missing value is too high, let's drop these feature (but not 'noParkSpaces')

columns_to_drop = ['telekomHybridUploadSpeed','electricityKwhPrice','electricityBasePrice']
df_berlin_cleaned = df_berlin.drop(columns=columns_to_drop)
df_berlin_cleaned.shape

(10406, 46)

In [14]:
df_berlin_cleaned.columns

Index(['regio1', 'serviceCharge', 'heatingType', 'telekomTvOffer',
       'newlyConst', 'balcony', 'picturecount', 'pricetrend',
       'telekomUploadSpeed', 'totalRent', 'yearConstructed', 'scoutId',
       'noParkSpaces', 'firingTypes', 'hasKitchen', 'geo_bln', 'cellar',
       'yearConstructedRange', 'baseRent', 'houseNumber', 'livingSpace',
       'geo_krs', 'condition', 'interiorQual', 'petsAllowed', 'street',
       'streetPlain', 'lift', 'baseRentRange', 'typeOfFlat', 'geo_plz',
       'noRooms', 'thermalChar', 'floor', 'numberOfFloors', 'noRoomsRange',
       'garden', 'livingSpaceRange', 'regio2', 'regio3', 'description',
       'facilities', 'heatingCosts', 'energyEfficiencyClass', 'lastRefurbish',
       'date'],
      dtype='object')


**In the following additional columns will be dropped off. This is based on the columns description on:** https://www.kaggle.com/datasets/corrieaar/apartment-rental-offers-in-germany?select=immo_data.csv

- **telekomTvOffer**: Is payed TV included if so which offer
- **picturecount**: how many pictures were uploaded to the listing
- **pricetrend**: price trend as calculated by Immoscout
- **firingTypes**: Team decided to drop
- **geo_bln**: bundesland (state), same as regio1 (Berlin)
- **geo_krs**: district, above ZIP code (unique value is Berlin, so same as regio1)
- **street**: street name (Duplicated with streetPlain, which has clearer formating)
- **interiorQual**: Team decided to drop
- **baseRentRange**: binned base rent, 1 to 9
- **thermalChar**: Team decided to drop
- **noRoomsRange**: binned number of rooms, 1 to 5
- **livingSpaceRange**: binned living space, 1 to 7
- **yearConstructedRange**: binned
- **regio2**: District or Kreis, same as geo krs (unique value is Berlin, so same as regio1)
- **energyEfficiencyClass**: Team decided to drop, since we have 
- **lastRefurbish**: Team decided to drop, since have yearConstructed & condition AND around 70% of the properties are missing value
- **date**: time of scraping




In [15]:
df_berlin_cleaned['numberOfFloors'].unique()

array([ 3.,  5.,  7.,  6.,  2., nan,  4., 14., 11.,  8., 12.,  0.,  9.,
       10.,  1., 16., 17., 18., 15., 21., 13., 19., 20., 25., 22., 27.])

In [16]:
#Dropping off columns which are not relevant for our project based on column description

columns_to_drop_2 = ['telekomTvOffer','picturecount','pricetrend','firingTypes','geo_bln','geo_krs','street'\
                     ,'interiorQual','baseRentRange','thermalChar','noRoomsRange','yearConstructedRange','livingSpaceRange','regio2','energyEfficiencyClass','lastRefurbish','date']
df_berlin_cleaned = df_berlin_cleaned.drop(columns=columns_to_drop_2)
df_berlin_cleaned.shape

(10406, 29)

## Dropping rows/properties

In [17]:
#Overview of remaining features and its % missing value

df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

noParkSpaces          0.806554
petsAllowed           0.464924
heatingCosts          0.458293
numberOfFloors        0.300500
condition             0.217951
streetPlain           0.146742
houseNumber           0.146742
heatingType           0.141361
yearConstructed       0.136940
telekomUploadSpeed    0.135787
facilities            0.127138
floor                 0.105708
typeOfFlat            0.077263
description           0.063809
totalRent             0.063617
serviceCharge         0.022583
geo_plz               0.000000
regio3                0.000000
noRooms               0.000000
garden                0.000000
regio1                0.000000
lift                  0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
balcony               0.000000
newlyConst            0.000000
livingSpace           0.000000
dtype: float64

### Drop properties with missing address

**1527 Properties (15%) are missing street information**. Therefore, we should drop these properties.

Berlin total = 10.406 properties \
Berlin missing address = 1.527 properties\
Berlin with address = 8.879 properties

In [18]:
#number of properties missing street info

df_berlin_cleaned['streetPlain'].isnull().sum()

1527

In [19]:
#list of properties with missing address. We need to drop these!

berlin_properties_no_street = df_berlin_cleaned[df_berlin_cleaned['streetPlain'].isnull()] #.value_counts()
berlin_properties_no_street

,regio1,serviceCharge,heatingType,newlyConst,balcony,telekomUploadSpeed,totalRent,yearConstructed,scoutId,noParkSpaces,...,typeOfFlat,geo_plz,noRooms,floor,numberOfFloors,garden,regio3,description,facilities,heatingCosts
286,Berlin,215.00,central_heating,False,True,10.0,1125.00,1890.0,110747740,NaN,...,apartment,13127,3.5,1.0,2.0,False,Französisch_Buchholz_Pankow,"Das Haus befindet sich an einer Hauptstraße, d...",Die Wohnung befindet sich in einem kleinen Meh...,118.0
300,Berlin,450.00,floor_heating,True,True,40.0,2830.00,2018.0,111780700,NaN,...,penthouse,10589,3.0,6.0,6.0,False,Charlottenburg_Charlottenburg,"Die hier angebotene, exklusive Penthouse ist T...","•\tGroßzügiger, lichtdurchfluteter Wohnbereich...",NaN
547,Berlin,250.00,central_heating,False,True,40.0,1125.00,1990.0,105407548,1.0,...,apartment,14052,2.0,1.0,3.0,True,Charlottenburg_Charlottenburg,Das Wohnhaus - erbaut etwa 1990 - liegt in Cha...,Die Wohnräume und die Diele verfügen über eine...,90.0
883,Berlin,155.00,central_heating,False,True,40.0,1766.00,1914.0,115176177,NaN,...,roof_storey,10245,3.0,5.0,5.0,False,Friedrichshain_Friedrichshain,Suchen Sie eine 3-Zimmerwohnung in Friedrichsh...,NaN,111.0
1035,Berlin,225.00,central_heating,False,False,40.0,1215.00,1932.0,111328308,NaN,...,apartment,12527,3.0,2.0,3.0,False,Schmöckwitz_Köpenick,ERSTBEZUG NACH SANIERUNG!\n\nWir bieten Ihnen ...,- Erstbezug nach Sanierung\n- hochwertiges Ech...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268086,Berlin,231.55,central_heating,False,True,40.0,794.35,1974.0,64925144,NaN,...,raised_ground_floor,13589,2.0,0.0,1.0,False,Spandau_Spandau,Freistehendes Mehrfamilienhaus in ruhiger Seit...,Das Wohnzimmer ist mit Parkettboden ausgestatt...,NaN
268321,Berlin,150.00,central_heating,False,True,40.0,1795.00,NaN,114135846,1.0,...,roof_storey,10555,3.0,5.0,5.0,False,Tiergarten_Tiergarten,Das imposante Wohn-und Geschäftsgebäude aus de...,Eine schöne Dachterrasse mit fast 10 m² realer...,100.0
268470,Berlin,130.00,self_contained_central_heating,False,True,40.0,2895.00,1915.0,64618855,2.0,...,apartment,10119,4.5,2.0,5.0,False,Prenzlauer_Berg_Prenzlauer_Berg,"top sanierter Altbau, großzügige Innenhofsitua...",Der Fußboden ist mit einem Echtholz-Stabeichen...,70.0
268491,Berlin,390.00,oil_heating,False,True,40.0,2600.00,1898.0,115231273,NaN,...,apartment,14193,3.0,NaN,NaN,False,Grunewald_Wilmersdorf,"• Die gemütliche, geschmackvoll möblierte, woh...",• Öl-Zentralheizung mit zentraler Warmwasserve...,NaN


In [20]:
#Dropping properties without street info

df_berlin_cleaned = df_berlin_cleaned.dropna(subset=['streetPlain'])
df_berlin_cleaned.shape

(8879, 29)

 ### Drop properties with missing price???
 <font color='red'>Around 6% of remaining properties do not have price. Shoud we drop them off???</font> 
 
tbd

In [21]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

noParkSpaces          0.815632
petsAllowed           0.468634
heatingCosts          0.417051
numberOfFloors        0.305552
condition             0.211623
heatingType           0.144048
telekomUploadSpeed    0.138304
yearConstructed       0.132785
facilities            0.111499
floor                 0.095732
typeOfFlat            0.073319
description           0.066787
totalRent             0.057439
serviceCharge         0.017457
regio3                0.000000
garden                0.000000
noRooms               0.000000
geo_plz               0.000000
regio1                0.000000
lift                  0.000000
streetPlain           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
balcony               0.000000
newlyConst            0.000000
livingSpace           0.000000
dtype: float64

In [22]:
#...

## Replacing missing data

In the following we should replace/clear/construct features that have missing values.

### noParkSpaces

Around **80% of the properties** do not have information on number of parking spaces. 

Solution: To handle the 'noParkSpaces' column where 80% of the values are missing, we can **create an additional binary column 'ParkSpaces' indicating whether a property has parking spaces or not.**

Then DROP "noParkSpaces" AND KEEP 'ParkSpaces'
 
- 1 = property has parking space
- 0 = property has NO parking space


In [23]:
df_berlin_cleaned['noParkSpaces'].value_counts().sort_values(ascending=False)

1.0     1285
0.0      161
2.0       86
20.0      45
24.0      18
3.0        8
6.0        6
50.0       4
30.0       4
5.0        3
54.0       2
22.0       2
10.0       2
25.0       2
26.0       2
4.0        1
9.0        1
13.0       1
8.0        1
7.0        1
34.0       1
58.0       1
Name: noParkSpaces, dtype: int64

In [24]:
# Create a new binary column 'ParkSpaces'
df_berlin_cleaned['ParkSpaces'] = df_berlin_cleaned['noParkSpaces'].apply(lambda x: 1 if pd.notnull(x) else 0)

#Double check
df_berlin_cleaned[['noParkSpaces','ParkSpaces']]

,noParkSpaces,ParkSpaces
45,1.0,1
61,NaN,0
77,1.0,1
86,NaN,0
87,NaN,0
...,...,...
268709,1.0,1
268733,NaN,0
268746,NaN,0
268758,NaN,0


In [25]:
#First drop the column 'noParkSpaces' and then let's show an overview of the % of missing value for remaining columns

df_berlin_cleaned = df_berlin_cleaned.drop(columns=['noParkSpaces'])
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)
#df_berlin_cleaned_3.columns

petsAllowed           0.468634
heatingCosts          0.417051
numberOfFloors        0.305552
condition             0.211623
heatingType           0.144048
telekomUploadSpeed    0.138304
yearConstructed       0.132785
facilities            0.111499
floor                 0.095732
typeOfFlat            0.073319
description           0.066787
totalRent             0.057439
serviceCharge         0.017457
garden                0.000000
regio3                0.000000
noRooms               0.000000
geo_plz               0.000000
regio1                0.000000
lift                  0.000000
streetPlain           0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
balcony               0.000000
newlyConst            0.000000
ParkSpaces            0.000000
dtype: float64

### petsAllowed

Around **50% of the properties** do not have information on petsAllowed.

Solution: Team decided to replace missing data by 'no'

In [26]:
df_berlin_cleaned['petsAllowed'].value_counts()


negotiable    3490
no             994
yes            234
Name: petsAllowed, dtype: int64

In [27]:
#Check on how many properties do not have info if pets are allowed
df_berlin_cleaned['petsAllowed'].isnull().sum()

4161

In [28]:
#Replace NaN (null) values in the petsAllowed column with the string 'no'
df_berlin_cleaned['petsAllowed'] = df_berlin_cleaned['petsAllowed'].fillna('no')

#Recheck on how many properties do not have info if pets are allowed. If 0 then we successfully replaced NaN values
df_berlin_cleaned['petsAllowed'].isnull().sum()

0

In [29]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

heatingCosts          0.417051
numberOfFloors        0.305552
condition             0.211623
heatingType           0.144048
telekomUploadSpeed    0.138304
yearConstructed       0.132785
facilities            0.111499
floor                 0.095732
typeOfFlat            0.073319
description           0.066787
totalRent             0.057439
serviceCharge         0.017457
regio3                0.000000
garden                0.000000
lift                  0.000000
noRooms               0.000000
geo_plz               0.000000
regio1                0.000000
streetPlain           0.000000
petsAllowed           0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
balcony               0.000000
newlyConst            0.000000
ParkSpaces            0.000000
dtype: float64

### heatingCosts 

Around **40% of the properties** do not have information on monthly heating costs in €.

Solution: To fill the missing heatingCosts values based on the **average heating cost per room**, you can follow these steps:

1. Calculate the average heating cost per room for the rows where the heatingCosts is not missing.
2. Use this average to fill in the missing values by multiplying it by the noRooms for each property with missing heating costs.

In [30]:
# Calculate the average heating cost per room for non-missing values
df_with_heating_costs = df_berlin_cleaned.dropna(subset=['heatingCosts'])
average_cost_per_room = (df_with_heating_costs['heatingCosts'] / df_with_heating_costs['noRooms']).mean()

print(f"Average heating cost per room: {average_cost_per_room}")


Average heating cost per room: 35.126613090844536


In [31]:
# Define a function to fill missing heatingCosts based on the number of rooms
def fill_heating_cost(row):
    if pd.isnull(row['heatingCosts']):
        return row['noRooms'] * average_cost_per_room
    return row['heatingCosts']

# Apply the function to the DataFrame
df_berlin_cleaned['heatingCosts'] = df_berlin_cleaned.apply(fill_heating_cost, axis=1)

df_berlin_cleaned['heatingCosts'] = df_berlin_cleaned['heatingCosts'].astype(float).round(1)


In [32]:
#Recheck on how many properties do not have info if heating costs. If 0 then we successfully replaced NaN values
df_berlin_cleaned['heatingCosts'].isnull().sum()

0

In [33]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

numberOfFloors        0.305552
condition             0.211623
heatingType           0.144048
telekomUploadSpeed    0.138304
yearConstructed       0.132785
facilities            0.111499
floor                 0.095732
typeOfFlat            0.073319
description           0.066787
totalRent             0.057439
serviceCharge         0.017457
lift                  0.000000
regio3                0.000000
garden                0.000000
heatingCosts          0.000000
noRooms               0.000000
geo_plz               0.000000
regio1                0.000000
streetPlain           0.000000
petsAllowed           0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
balcony               0.000000
newlyConst            0.000000
ParkSpaces            0.000000
dtype: float64

### numberOfFloors 

Around **30% of the properties** do not have information on number of floors in the building.

Solution: Random Sampling from Known Distribution

In [34]:
# Get non-null values of numberOfFloors
known_floors = df_berlin_cleaned['numberOfFloors'].dropna()

# Fill missing values by sampling from the known distribution
missing_indices = df_berlin_cleaned[df_berlin_cleaned['numberOfFloors'].isnull()].index
df_berlin_cleaned.loc[missing_indices, 'numberOfFloors'] = np.random.choice(known_floors, size=len(missing_indices), replace=True)


In [35]:
#Recheck on how many properties do not have info if heating costs. If 0 then we successfully replaced NaN values
df_berlin_cleaned['numberOfFloors'].isnull().sum()

0

In [36]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

condition             0.211623
heatingType           0.144048
telekomUploadSpeed    0.138304
yearConstructed       0.132785
facilities            0.111499
floor                 0.095732
typeOfFlat            0.073319
description           0.066787
totalRent             0.057439
serviceCharge         0.017457
lift                  0.000000
regio3                0.000000
garden                0.000000
numberOfFloors        0.000000
heatingCosts          0.000000
noRooms               0.000000
geo_plz               0.000000
regio1                0.000000
streetPlain           0.000000
petsAllowed           0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
balcony               0.000000
newlyConst            0.000000
ParkSpaces            0.000000
dtype: float64

### condition 

Around **20% of the properties** do not have information on condition of the flat.

Solution: Fill with Mode

In [37]:
# Calculate the mode of the 'condition' column
mode_condition = df_berlin_cleaned['condition'].mode()[0]

# Fill missing values with the mode
df_berlin_cleaned['condition'].fillna(mode_condition, inplace=True)


In [38]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

heatingType           0.144048
telekomUploadSpeed    0.138304
yearConstructed       0.132785
facilities            0.111499
floor                 0.095732
typeOfFlat            0.073319
description           0.066787
totalRent             0.057439
serviceCharge         0.017457
lift                  0.000000
regio3                0.000000
garden                0.000000
numberOfFloors        0.000000
heatingCosts          0.000000
noRooms               0.000000
geo_plz               0.000000
regio1                0.000000
condition             0.000000
streetPlain           0.000000
petsAllowed           0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
balcony               0.000000
newlyConst            0.000000
ParkSpaces            0.000000
dtype: float64

### typeOfFlat  

Around **7% of the properties** do not have information on condition of typeOfFlat.

Solution: Fill with Mode

In [39]:
# Calculate the mode of the 'typeOfFlat' column
mode_flat_type = df_berlin_cleaned['typeOfFlat'].mode()[0]

# Fill missing values with the mode
df_berlin_cleaned['typeOfFlat'].fillna(mode_flat_type, inplace=True)



In [40]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

heatingType           0.144048
telekomUploadSpeed    0.138304
yearConstructed       0.132785
facilities            0.111499
floor                 0.095732
description           0.066787
totalRent             0.057439
serviceCharge         0.017457
numberOfFloors        0.000000
geo_plz               0.000000
noRooms               0.000000
regio1                0.000000
garden                0.000000
regio3                0.000000
lift                  0.000000
heatingCosts          0.000000
typeOfFlat            0.000000
condition             0.000000
streetPlain           0.000000
petsAllowed           0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
balcony               0.000000
newlyConst            0.000000
ParkSpaces            0.000000
dtype: float64

### heatingType 

Around **14% of the properties** do not have information on heatingType.

Solution: Fill missing 'heatingType' based on the mode of 'typeOfFlat'

In [41]:
# Fill missing 'heatingType' based on the mode of 'typeOfFlat'
def fill_heating_type(row):
    if pd.isnull(row['heatingType']):
        return df_berlin_cleaned[df_berlin_cleaned['typeOfFlat'] == row['typeOfFlat']]['heatingType'].mode()[0]
    return row['heatingType']

df_berlin_cleaned['heatingType'] = df_berlin_cleaned.apply(fill_heating_type, axis=1)


In [42]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

telekomUploadSpeed    0.138304
yearConstructed       0.132785
facilities            0.111499
floor                 0.095732
description           0.066787
totalRent             0.057439
serviceCharge         0.017457
regio1                0.000000
lift                  0.000000
heatingCosts          0.000000
regio3                0.000000
garden                0.000000
numberOfFloors        0.000000
noRooms               0.000000
geo_plz               0.000000
typeOfFlat            0.000000
condition             0.000000
streetPlain           0.000000
petsAllowed           0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
balcony               0.000000
newlyConst            0.000000
heatingType           0.000000
ParkSpaces            0.000000
dtype: float64

### telekomUploadSpeed 

Around **14% of the properties** do not have information on telekomUploadSpeed.

Solution: Fill with Mean


In [43]:
# Calculate mean and median of the 'telekomUploadSpeed' column
mean_upload_speed = df_berlin_cleaned['telekomUploadSpeed'].mean()

# Fill missing values with the mean or median
df_berlin_cleaned['telekomUploadSpeed'].fillna(mean_upload_speed, inplace=True)

df_berlin_cleaned['telekomUploadSpeed'] = df_berlin_cleaned['telekomUploadSpeed'].astype(float).round(1)

In [44]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

yearConstructed       0.132785
facilities            0.111499
floor                 0.095732
description           0.066787
totalRent             0.057439
serviceCharge         0.017457
regio1                0.000000
streetPlain           0.000000
heatingCosts          0.000000
regio3                0.000000
garden                0.000000
numberOfFloors        0.000000
noRooms               0.000000
geo_plz               0.000000
typeOfFlat            0.000000
lift                  0.000000
condition             0.000000
petsAllowed           0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
telekomUploadSpeed    0.000000
balcony               0.000000
newlyConst            0.000000
heatingType           0.000000
ParkSpaces            0.000000
dtype: float64

In [45]:
df_berlin_cleaned[['telekomUploadSpeed']]

,telekomUploadSpeed
45,32.1
61,40.0
77,40.0
86,32.1
87,40.0
...,...
268709,10.0
268733,40.0
268746,40.0
268758,40.0


### yearConstructed

Around **14% of the properties** do not have information on yearConstructed.

Solution: Calculate median of the 'yearConstructed' column


In [46]:
# Calculate mean and median of the 'yearConstructed' column
median_year = df_berlin_cleaned['yearConstructed'].median()

# Fill missing values with the mean or median
df_berlin_cleaned['yearConstructed'].fillna(median_year, inplace=True)  # Using median for robustness


In [47]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

facilities            0.111499
floor                 0.095732
description           0.066787
totalRent             0.057439
serviceCharge         0.017457
regio1                0.000000
streetPlain           0.000000
heatingCosts          0.000000
regio3                0.000000
garden                0.000000
numberOfFloors        0.000000
noRooms               0.000000
geo_plz               0.000000
typeOfFlat            0.000000
lift                  0.000000
condition             0.000000
petsAllowed           0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
yearConstructed       0.000000
telekomUploadSpeed    0.000000
balcony               0.000000
newlyConst            0.000000
heatingType           0.000000
ParkSpaces            0.000000
dtype: float64

### floor

Around **10% of the properties** do not have information on which floor is the flat on.

Solution: Randomly assign a floor number within the range of 1 to numberOfFloors for missing values.

In [48]:
df_berlin_cleaned['numberOfFloors'].unique()

array([ 3.,  5.,  7.,  6.,  2.,  4., 14., 11., 10.,  8., 12.,  0.,  9.,
        1., 16., 17., 18., 15., 21., 13., 19., 22., 20., 25., 27.])

In [49]:
# Function to fill missing 'floor' with a random value within the range
def assign_random_floor(row):
    if pd.isnull(row['floor']):
        return np.random.randint(0, row['numberOfFloors'] + 1)
    return row['floor']

df_berlin_cleaned['floor'] = df_berlin_cleaned.apply(assign_random_floor, axis=1)


In [50]:
df_berlin_cleaned.rename(columns={'floor': 'floorNo'}, inplace=True)

In [51]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

facilities            0.111499
description           0.066787
totalRent             0.057439
serviceCharge         0.017457
regio1                0.000000
streetPlain           0.000000
heatingCosts          0.000000
regio3                0.000000
garden                0.000000
numberOfFloors        0.000000
floorNo               0.000000
noRooms               0.000000
geo_plz               0.000000
typeOfFlat            0.000000
lift                  0.000000
condition             0.000000
petsAllowed           0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
yearConstructed       0.000000
telekomUploadSpeed    0.000000
balcony               0.000000
newlyConst            0.000000
heatingType           0.000000
ParkSpaces            0.000000
dtype: float64

### serviceCharge  

Around **2% of the properties** do not have information on serviceCharge.

Solution: Fill missing values based on the mean or median serviceCharge within groups of similar properties, such as by typeOfFlat, geo_plz, or other relevant features.

In [52]:
# Calculate median
median_serviceCharge = df_berlin_cleaned['serviceCharge'].median()

# Calculate median serviceCharge by typeOfFlat (or another relevant grouping)
median_serviceCharge_by_group = df_berlin_cleaned.groupby('typeOfFlat')['serviceCharge'].median()

# Function to fill missing serviceCharge based on group median
def fill_serviceCharge(row):
    if pd.isnull(row['serviceCharge']):
        return median_serviceCharge_by_group.get(row['typeOfFlat'], median_serviceCharge)  # Default to overall median if group is missing
    return row['serviceCharge']

df_berlin_cleaned['serviceCharge'] = df_berlin_cleaned.apply(fill_serviceCharge, axis=1)


In [53]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

facilities            0.111499
description           0.066787
totalRent             0.057439
regio1                0.000000
petsAllowed           0.000000
heatingCosts          0.000000
regio3                0.000000
garden                0.000000
numberOfFloors        0.000000
floorNo               0.000000
noRooms               0.000000
geo_plz               0.000000
typeOfFlat            0.000000
lift                  0.000000
streetPlain           0.000000
condition             0.000000
serviceCharge         0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
yearConstructed       0.000000
telekomUploadSpeed    0.000000
balcony               0.000000
newlyConst            0.000000
heatingType           0.000000
ParkSpaces            0.000000
dtype: float64

### totalRent  

Around **6% of the properties** do not have information on total rent (usually a sum of base rent, service charge and heating cost).

Solution: Sum of of base rent, service charge and heating cost

In [54]:
df_berlin_cleaned['totalRent'] = df_berlin_cleaned.apply(
    lambda row: row['baseRent'] + row['serviceCharge'] + row['heatingCosts']
    if pd.isnull(row['totalRent'])
    else row['totalRent'],
    axis=1
)

In [55]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

facilities            0.111499
description           0.066787
regio1                0.000000
petsAllowed           0.000000
heatingCosts          0.000000
regio3                0.000000
garden                0.000000
numberOfFloors        0.000000
floorNo               0.000000
noRooms               0.000000
geo_plz               0.000000
typeOfFlat            0.000000
lift                  0.000000
streetPlain           0.000000
condition             0.000000
serviceCharge         0.000000
livingSpace           0.000000
houseNumber           0.000000
baseRent              0.000000
cellar                0.000000
hasKitchen            0.000000
scoutId               0.000000
yearConstructed       0.000000
totalRent             0.000000
telekomUploadSpeed    0.000000
balcony               0.000000
newlyConst            0.000000
heatingType           0.000000
ParkSpaces            0.000000
dtype: float64

### facilities & description

Around **11% of the properties** do not have information discribtion.

Solution: Replace empty value with 'na'


In [56]:
df_berlin_cleaned['facilities'].fillna('na', inplace=True)
df_berlin_cleaned['description'].fillna('na', inplace=True)


In [57]:
df_berlin_cleaned.isnull().sum().sort_values(ascending=False)/len(df_berlin_cleaned)

regio1                0.0
petsAllowed           0.0
heatingCosts          0.0
facilities            0.0
description           0.0
regio3                0.0
garden                0.0
numberOfFloors        0.0
floorNo               0.0
noRooms               0.0
geo_plz               0.0
typeOfFlat            0.0
lift                  0.0
streetPlain           0.0
condition             0.0
serviceCharge         0.0
livingSpace           0.0
houseNumber           0.0
baseRent              0.0
cellar                0.0
hasKitchen            0.0
scoutId               0.0
yearConstructed       0.0
totalRent             0.0
telekomUploadSpeed    0.0
balcony               0.0
newlyConst            0.0
heatingType           0.0
ParkSpaces            0.0
dtype: float64

### newlyConst, balcony, lift, hasKitchen cellar & garden

Replace values:
- TRUE => 1
- FALSE => 0

In [58]:
df_berlin_cleaned['newlyConst'] = df_berlin_cleaned['newlyConst'].replace({True: 1, False: 0})
df_berlin_cleaned['balcony'] = df_berlin_cleaned['balcony'].replace({True: 1, False: 0})
df_berlin_cleaned['lift'] = df_berlin_cleaned['lift'].replace({True: 1, False: 0})
df_berlin_cleaned['hasKitchen'] = df_berlin_cleaned['hasKitchen'].replace({True: 1, False: 0})
df_berlin_cleaned['cellar'] = df_berlin_cleaned['hasKitchen'].replace({True: 1, False: 0})
df_berlin_cleaned['garden'] = df_berlin_cleaned['garden'].replace({True: 1, False: 0})


### petsAllowed

Replace value:
- yes => 1
- no => 2
- negotiable => 3

In [59]:
df_berlin_cleaned['petsAllowed'] = df_berlin_cleaned['petsAllowed'].replace({'yes': 1, 'no': 2, 'negotiable': 3})

# Creating Full Address column for API call

Create a new column **'fullAddress'** that serves as **input for an API Overpass** call. We need to concatenate the values from the houseNumber, streetPlain, geo_plz, regio1. Additionally we need to include the country in the address string. \
PS: country column is not yet in the dataset

1. Replace '_' in the streetPlain by ' '
2. Merge houseNumber, streetPlain, geo_pl and regio1, country

DROP houseNumber, streetPlain, geo_plz, regio1. & KEEP fullAddress only


In [60]:
#First: Remove underscores from 'streetPlain' column
df_berlin_cleaned['streetPlain'] = df_berlin_cleaned['streetPlain'].str.replace('_', ' ')

In [61]:
#Second: Create the new column for Overpass API input including the country
df_berlin_cleaned['fullAddress'] = df_berlin_cleaned.apply(
    lambda row: f"{row['streetPlain']} {row['houseNumber']}, {row['geo_plz']} {row['regio1']}, Germany", axis=1)
df_berlin_cleaned[['streetPlain','houseNumber','geo_plz', 'regio1','fullAddress']]

,streetPlain,houseNumber,geo_plz,regio1,fullAddress
45,Metropolitan Park,63,13591,Berlin,"Metropolitan Park 63, 13591 Berlin, Germany"
61,Börnestraße,11,13086,Berlin,"Börnestraße 11, 13086 Berlin, Germany"
77,Stallschreiberstraße,27,10179,Berlin,"Stallschreiberstraße 27, 10179 Berlin, Germany"
86,Hallesche Straße,5a,10963,Berlin,"Hallesche Straße 5a, 10963 Berlin, Germany"
87,Heidestraße,19,10557,Berlin,"Heidestraße 19, 10557 Berlin, Germany"
...,...,...,...,...,...
268709,Wartburgstraße,53a,10823,Berlin,"Wartburgstraße 53a, 10823 Berlin, Germany"
268733,Brüderstraße,22,13595,Berlin,"Brüderstraße 22, 13595 Berlin, Germany"
268746,Todtnauer Zeile,3,13469,Berlin,"Todtnauer Zeile 3, 13469 Berlin, Germany"
268758,Leibnizstr.,85,10625,Berlin,"Leibnizstr. 85, 10625 Berlin, Germany"


In [62]:
# Drop the specified columns
df_berlin_cleaned.drop(columns=['houseNumber', 'streetPlain', 'geo_plz', 'regio1'], inplace=True)


In [63]:
#Let's check on the status of total columns
df_berlin_cleaned.columns

Index(['serviceCharge', 'heatingType', 'newlyConst', 'balcony',
       'telekomUploadSpeed', 'totalRent', 'yearConstructed', 'scoutId',
       'hasKitchen', 'cellar', 'baseRent', 'livingSpace', 'condition',
       'petsAllowed', 'lift', 'typeOfFlat', 'noRooms', 'floorNo',
       'numberOfFloors', 'garden', 'regio3', 'description', 'facilities',
       'heatingCosts', 'ParkSpaces', 'fullAddress'],
      dtype='object')

In [64]:
df_berlin_cleaned.head(10)

,serviceCharge,heatingType,newlyConst,balcony,telekomUploadSpeed,totalRent,yearConstructed,scoutId,hasKitchen,cellar,...,noRooms,floorNo,numberOfFloors,garden,regio3,description,facilities,heatingCosts,ParkSpaces,fullAddress
45,320.00,central_heating,0,1,32.1,1140.00,1980.0,115671495,1,1,...,3.0,0.0,3.0,0,Staaken_Spandau,Diese attraktive EG-Wohnung kann ab sofort bez...,na,105.4,1,"Metropolitan Park 63, 13591 Berlin, Germany"
61,79.00,central_heating,0,0,40.0,955.00,1918.0,113359456,0,0,...,2.0,0.0,3.0,0,Weißensee_Weißensee,Suchen Sie eine 2-Zimmerwohnung in Weißensee? ...,Zusammengefasst bietet die Wohnung:\n+ Vollstä...,68.0,0,"Börnestraße 11, 13086 Berlin, Germany"
77,150.00,floor_heating,1,1,40.0,1300.00,2019.0,113704695,1,1,...,2.0,3.0,5.0,0,Mitte_Mitte,Dieses luxuriöse Neubauquartier direkt am Luis...,"• sehr ruhig gelegen, trotz zentraler Lage, da...",70.3,1,"Stallschreiberstraße 27, 10179 Berlin, Germany"
86,228.78,floor_heating,0,1,32.1,1428.78,2017.0,107589685,1,1,...,2.5,6.0,7.0,0,Kreuzberg_Kreuzberg,Bei dieser ansprechenden Immobilie handelt es ...,Balkon und 10m Terasse,87.8,0,"Hallesche Straße 5a, 10963 Berlin, Germany"
87,147.08,district_heating,1,1,40.0,1559.05,2019.0,114773727,1,1,...,2.0,0.0,6.0,0,Tiergarten_Tiergarten,"In the """"Wasserstadt - Mitte"""" you can expect ...",Your spacious ground floor apartment is equipp...,73.5,0,"Heidestraße 19, 10557 Berlin, Germany"
126,232.00,combined_heat_and_power_plant,0,1,40.0,1280.00,2014.0,113764604,1,1,...,3.0,1.0,2.0,0,Grünau_Köpenick,Direkt am Waldrand - Berlin Grünau\n\n- wunder...,Keine Wünsche offen....& an alles gedacht........,120.0,1,"Waldkiefernring 5, 12527 Berlin, Germany"
142,221.11,district_heating,0,1,40.0,656.72,1980.0,113829068,0,0,...,3.0,16.0,6.0,0,Marzahn_Marzahn,Die Punkthochhäuser der Bärensteinstraße wurde...,Von der Wohnung aus dem 17. Stock genießen Sie...,74.6,0,"Bärensteinstr. 42, 12685 Berlin, Germany"
155,105.00,gas_heating,0,1,40.0,1155.00,1870.0,84867476,1,1,...,4.0,1.0,2.0,1,Köpenick_Köpenick,Das komplette Haus wurde 2003 Denkmal gerecht ...,Küche: Hochwertige Einbauküche mit Geschirrspü...,140.0,0,"Grünauer Str. 22, 12557 Berlin, Germany"
187,162.00,district_heating,0,1,40.0,861.00,1984.0,113297201,1,1,...,3.0,4.0,5.0,0,Neu_Hohenschönhausen_Hohenschönhausen,Die Objekte wurden in den 80er Jahren errichte...,- Balkon\r- PVC Bodenbelag\r- Küche mit Fenste...,80.0,0,"Falkenberger Chaussee 130, 13057 Berlin, Germany"
194,67.15,central_heating,0,0,40.0,415.51,1988.0,112185558,0,0,...,1.0,2.0,5.0,0,Hellersdorf_Hellersdorf,"Der Siedlungshof, in dem sich Ihre neue Wohnun...",saniert:\n* modern gefliestes Bad mit Dusche\n...,33.6,0,"Luckenwalder Str. 94, 12629 Berlin, Germany"


In [65]:
df_to_save = df_berlin_cleaned.drop(['description', 'facilities'], axis=1)

In [66]:
df_to_save.to_csv('berlin_cleaned.csv')

In [67]:
df_test = pd.read_csv('berlin_cleaned.csv')
df_test.shape

(8879, 25)

In [68]:
df_test.head()

,Unnamed: 0,serviceCharge,heatingType,newlyConst,balcony,telekomUploadSpeed,totalRent,yearConstructed,scoutId,hasKitchen,...,lift,typeOfFlat,noRooms,floorNo,numberOfFloors,garden,regio3,heatingCosts,ParkSpaces,fullAddress
0,45,320.00,central_heating,0,1,32.1,1140.00,1980.0,115671495,1,...,1,ground_floor,3.0,0.0,3.0,0,Staaken_Spandau,105.4,1,"Metropolitan Park 63, 13591 Berlin, Germany"
1,61,79.00,central_heating,0,0,40.0,955.00,1918.0,113359456,0,...,0,ground_floor,2.0,0.0,3.0,0,Weißensee_Weißensee,68.0,0,"Börnestraße 11, 13086 Berlin, Germany"
2,77,150.00,floor_heating,1,1,40.0,1300.00,2019.0,113704695,1,...,1,apartment,2.0,3.0,5.0,0,Mitte_Mitte,70.3,1,"Stallschreiberstraße 27, 10179 Berlin, Germany"
3,86,228.78,floor_heating,0,1,32.1,1428.78,2017.0,107589685,1,...,1,apartment,2.5,6.0,7.0,0,Kreuzberg_Kreuzberg,87.8,0,"Hallesche Straße 5a, 10963 Berlin, Germany"
4,87,147.08,district_heating,1,1,40.0,1559.05,2019.0,114773727,1,...,1,ground_floor,2.0,0.0,6.0,0,Tiergarten_Tiergarten,73.5,0,"Heidestraße 19, 10557 Berlin, Germany"


In [69]:
df_test = df_test.drop(columns='Unnamed: 0')
df_test

,serviceCharge,heatingType,newlyConst,balcony,telekomUploadSpeed,totalRent,yearConstructed,scoutId,hasKitchen,cellar,...,lift,typeOfFlat,noRooms,floorNo,numberOfFloors,garden,regio3,heatingCosts,ParkSpaces,fullAddress
0,320.00,central_heating,0,1,32.1,1140.00,1980.0,115671495,1,1,...,1,ground_floor,3.0,0.0,3.0,0,Staaken_Spandau,105.4,1,"Metropolitan Park 63, 13591 Berlin, Germany"
1,79.00,central_heating,0,0,40.0,955.00,1918.0,113359456,0,0,...,0,ground_floor,2.0,0.0,3.0,0,Weißensee_Weißensee,68.0,0,"Börnestraße 11, 13086 Berlin, Germany"
2,150.00,floor_heating,1,1,40.0,1300.00,2019.0,113704695,1,1,...,1,apartment,2.0,3.0,5.0,0,Mitte_Mitte,70.3,1,"Stallschreiberstraße 27, 10179 Berlin, Germany"
3,228.78,floor_heating,0,1,32.1,1428.78,2017.0,107589685,1,1,...,1,apartment,2.5,6.0,7.0,0,Kreuzberg_Kreuzberg,87.8,0,"Hallesche Straße 5a, 10963 Berlin, Germany"
4,147.08,district_heating,1,1,40.0,1559.05,2019.0,114773727,1,1,...,1,ground_floor,2.0,0.0,6.0,0,Tiergarten_Tiergarten,73.5,0,"Heidestraße 19, 10557 Berlin, Germany"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8874,150.00,central_heating,0,1,10.0,1250.00,1996.0,82368044,1,1,...,1,apartment,3.0,2.0,7.0,1,Schöneberg_Schöneberg,100.0,1,"Wartburgstraße 53a, 10823 Berlin, Germany"
8875,100.00,self_contained_central_heating,0,0,40.0,850.00,1910.0,106510759,1,1,...,0,apartment,2.0,3.0,5.0,0,Spandau_Spandau,70.3,0,"Brüderstraße 22, 13595 Berlin, Germany"
8876,200.00,district_heating,0,1,40.0,949.00,1967.0,110770132,0,0,...,1,apartment,3.5,2.0,4.0,0,Waidmannslust_Reinickendorf,130.0,0,"Todtnauer Zeile 3, 13469 Berlin, Germany"
8877,100.00,district_heating,1,0,40.0,919.00,2019.0,113649920,1,1,...,1,apartment,1.0,2.0,6.0,1,Charlottenburg_Charlottenburg,35.1,0,"Leibnizstr. 85, 10625 Berlin, Germany"


In [70]:
df_test.to_csv('berlin_cleaned.csv')